# PRUEBA API AEMET

## Imports

In [2]:
import requests
import math
import pandas as pd
from IPython.display import HTML, display
from datetime import datetime

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from dotenv import load_dotenv
import os

## Variables globales

In [3]:
# Cargar las variables del archivo .env
load_dotenv()

# AEMET API Key
AEMET_API_KEY = os.getenv("AEMET_API_KEY")

municipio = "Benalua"

## Lectura código de municipio

In [4]:
# Leer diccionario de municipios de la página de AEMET
df = pd.read_excel("data/diccionario24.xlsx", header=1, usecols=["CPRO","CMUN","NOMBRE"])

In [5]:
#  Filtrar por nombre
row = df[df["NOMBRE"].str.lower() == "benalúa"].iloc[0]

cod_prov = row["CPRO"]
cod_muni = row["CMUN"]

# codigo municipio = cod_prov + cod_muni
cod_muni = f"{cod_prov:02d}{cod_muni:03d}"

print(f"El código de municipio de {municipio} es: {cod_muni}")

El código de municipio de Benalua es: 18027


## Predicción de la semana siguiente

### Obtención de datos

In [7]:
# Consultar predicción diaria para el municipio
url_pred = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/diaria/{cod_muni}/"
headers = {"api_key": AEMET_API_KEY}

print(f"\nConsultando predicción diaria para municipio {cod_muni} ({municipio})")
r_pred = requests.get(url_pred, headers=headers)
r_pred.raise_for_status()
resp = r_pred.json()


Consultando predicción diaria para municipio 18027 (Benalua)


In [8]:
resp

{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/367ae770',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/dfd88b22'}

In [9]:
# Seguir enlace de datos si es posible
data_url = resp.get("datos")
if data_url:
    r_data = requests.get(data_url, headers=headers)
    r_data.raise_for_status()
    pred_data = r_data.json()
else:
    pred_data = resp

In [10]:
pred_data

[{'origen': {'productor': 'Agencia Estatal de Meteorología - AEMET. Gobierno de España',
   'web': 'https://www.aemet.es',
   'enlace': 'https://www.aemet.es/es/eltiempo/prediccion/municipios/benalua-id18027',
   'language': 'es',
   'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
   'notaLegal': 'https://www.aemet.es/es/nota_legal'},
  'elaborado': '2025-07-26T14:28:09',
  'nombre': 'Benalúa',
  'provincia': 'Granada',
  'prediccion': {'dia': [{'probPrecipitacion': [{'value': 0,
       'periodo': '00-24'},
      {'value': 0, 'periodo': '00-12'},
      {'value': 0, 'periodo': '12-24'},
      {'value': 0, 'periodo': '00-06'},
      {'value': 0, 'periodo': '06-12'},
      {'value': 0, 'periodo': '12-18'},
      {'value': 0, 'periodo': '18-24'}],
     'cotaNieveProv': [{'value': '', 'periodo': '00-24'},
      {'value': '', 'periodo': '00-12'},
      {'value': '', 'periodo': '12-24'},
      {'value': '', 'periodo': '00

### Formateo de datos

In [11]:
# Crear una lista de días
dias = pred_data[0]["prediccion"].get("dia", [])

dias

[{'probPrecipitacion': [{'value': 0, 'periodo': '00-24'},
   {'value': 0, 'periodo': '00-12'},
   {'value': 0, 'periodo': '12-24'},
   {'value': 0, 'periodo': '00-06'},
   {'value': 0, 'periodo': '06-12'},
   {'value': 0, 'periodo': '12-18'},
   {'value': 0, 'periodo': '18-24'}],
  'cotaNieveProv': [{'value': '', 'periodo': '00-24'},
   {'value': '', 'periodo': '00-12'},
   {'value': '', 'periodo': '12-24'},
   {'value': '', 'periodo': '00-06'},
   {'value': '', 'periodo': '06-12'},
   {'value': '', 'periodo': '12-18'},
   {'value': '', 'periodo': '18-24'}],
  'estadoCielo': [{'value': '', 'periodo': '00-24', 'descripcion': ''},
   {'value': '', 'periodo': '00-12', 'descripcion': ''},
   {'value': '12', 'periodo': '12-24', 'descripcion': 'Poco nuboso'},
   {'value': '', 'periodo': '00-06', 'descripcion': ''},
   {'value': '11', 'periodo': '06-12', 'descripcion': 'Despejado'},
   {'value': '12', 'periodo': '12-18', 'descripcion': 'Poco nuboso'},
   {'value': '12', 'periodo': '18-24', 'd

In [12]:
# Lista donde iremos guardando cada fila
filas = []

print(f"\nPredicción diaria para municipio {cod_muni} ({municipio}):")
for dia in dias:
    print("========================================")
    print(dia)
    # Fecha
    fecha_raw = dia.get("fecha", "")
    fecha = fecha_raw.split("T")[0] if "T" in fecha_raw else fecha_raw

    # Temperaturas
    tmax = dia.get("temperatura", {}).get("maxima")
    tmin = dia.get("temperatura", {}).get("minima")

    # Estado de cielo
    estado = dia.get("estadoCielo", [{}])[0].get("descripcion") or "Sin datos"
    print(f"{fecha}: {tmin}–{tmax} °C, {estado}")

    # Viento
    viento = dia.get("viento", [{}])[0]
    vel_viento = viento.get("velocidad")
    dir_viento = viento.get("direccion")

    # Humedad relativa
    humedad = dia.get("humedadRelativa", [{}])
    hummax = humedad.get("maxima")
    hummin = humedad.get("minima")

    # Probabilidad de lluvia
    prob_lluvia = dia.get("probPrecipitacion", [{}])[0].get("value", "Sin datos")

    # Añadimos el dict a la lista
    filas.append({
        "fecha": fecha,
        "tmax": tmax,
        "tmin": tmin,
        "estadoCielo": estado,
        "vel_viento": vel_viento,
        "dir_viento": dir_viento,
        "hummax": hummax,
        "hummin": hummin,
        "prob_lluvia": prob_lluvia
    })




Predicción diaria para municipio 18027 (Benalua):
{'probPrecipitacion': [{'value': 0, 'periodo': '00-24'}, {'value': 0, 'periodo': '00-12'}, {'value': 0, 'periodo': '12-24'}, {'value': 0, 'periodo': '00-06'}, {'value': 0, 'periodo': '06-12'}, {'value': 0, 'periodo': '12-18'}, {'value': 0, 'periodo': '18-24'}], 'cotaNieveProv': [{'value': '', 'periodo': '00-24'}, {'value': '', 'periodo': '00-12'}, {'value': '', 'periodo': '12-24'}, {'value': '', 'periodo': '00-06'}, {'value': '', 'periodo': '06-12'}, {'value': '', 'periodo': '12-18'}, {'value': '', 'periodo': '18-24'}], 'estadoCielo': [{'value': '', 'periodo': '00-24', 'descripcion': ''}, {'value': '', 'periodo': '00-12', 'descripcion': ''}, {'value': '12', 'periodo': '12-24', 'descripcion': 'Poco nuboso'}, {'value': '', 'periodo': '00-06', 'descripcion': ''}, {'value': '11', 'periodo': '06-12', 'descripcion': 'Despejado'}, {'value': '12', 'periodo': '12-18', 'descripcion': 'Poco nuboso'}, {'value': '12', 'periodo': '18-24', 'descripci

In [13]:
# Creamos el DataFrame de una sola vez
clima_semana = pd.DataFrame(filas)

# Mostrar DataFrame final
if not clima_semana.empty:
    print("\nClima semanal:")
    print(clima_semana)


Clima semanal:
        fecha  tmax  tmin         estadoCielo  vel_viento dir_viento  hummax  \
0  2025-07-26    35    17           Sin datos           0                 85   
1  2025-07-27    37    19           Despejado          10         NO      65   
2  2025-07-28    38    20           Despejado           5          N      55   
3  2025-07-29    35    18         Poco nuboso          10          N      85   
4  2025-07-30    34    17         Poco nuboso          10         NE      90   
5  2025-07-31    35    17  Intervalos nubosos          15          N      65   
6  2025-08-01    36    18  Intervalos nubosos           0          C      60   

   hummin  prob_lluvia  
0      20            0  
1      15            0  
2      10            0  
3      20            5  
4      25            5  
5      15           20  
6      20           25  


Los datos para el día actual vienen un poco raros, vamos a obtenerlos de otra forma.

In [15]:
# Consulta predicción horaria
url_pred = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/horaria/{cod_muni}/"
headers = {"api_key": AEMET_API_KEY}

print(f"\nConsultando predicción horaria para municipio {cod_muni} ({municipio})")
r_pred = requests.get(url_pred, headers=headers)
r_pred.raise_for_status()
resp = r_pred.json()



Consultando predicción horaria para municipio 18027 (Benalua)


In [16]:
# Seguir enlace de datos si se puede
data_url = resp.get("datos")
if data_url:
    r_data = requests.get(data_url, headers=headers)
    r_data.raise_for_status()
    pred_data_horaria = r_data.json()
else:
    pred_data_horaria = resp

In [17]:
# Crear lista de dias
info_horaria = pred_data_horaria[0]["prediccion"].get("dia", [])

info_horaria

[{'estadoCielo': [{'value': '11', 'periodo': '08', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '09', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '10', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '11', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '12', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '13', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '14', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '15', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '16', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '17', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '18', 'descripcion': 'Despejado'},
   {'value': '12', 'periodo': '19', 'descripcion': 'Poco nuboso'},
   {'value': '11', 'periodo': '20', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '21', 'descripcion': 'Despejado'},
   {'value': '11n', 'periodo': '22', 'descripcion': 'Despejado'},
   {'va

Extraemos las métricas del día de hoy pero solamente a partir de la hora actual

In [18]:
hoy = info_horaria[0]

hora_actual = datetime.now().hour 

# Extraer sólo los datos horarios >= hora actual
def filtrar_por_hora(lista_dicts, key_periodo='periodo'):
    return [d for d in lista_dicts if int(d.get(key_periodo, 0)) >= hora_actual]

temps = filtrar_por_hora(hoy['temperatura'])
cielo = filtrar_por_hora(hoy['estadoCielo'])
prec = filtrar_por_hora(hoy['precipitacion'])
hum = filtrar_por_hora(hoy['humedadRelativa'])
vty = filtrar_por_hora(hoy['vientoAndRachaMax'])

# Temperaturas máximas y mínimas
valores_t = [int(d['value']) for d in temps]
tmax = max(valores_t) if valores_t else None
tmin = min(valores_t) if valores_t else None

# Descripción del cielo
estado = cielo[0]['descripcion'] if cielo else 'Sin datos'

# Precipitación total
prec_vals = [float(d['value']) for d in prec]
rain_sum = sum(prec_vals)

# Humedad relativa min/max
hum_vals = [int(d['value']) for d in hum]
hummax = max(hum_vals) if hum_vals else None
hummin = min(hum_vals) if hum_vals else None

# Probabilidad de lluvia
prob_prec = hoy.get('probPrecipitacion', [])

# el periodo viene en un formato un poco raro, como '0208','0814'
# suponemos que la hora de inicio es el primer par de dígitos
pp_filtrada = [
    int(d['value'])
    for d in prob_prec
    if int(d.get('periodo','0000')[:2]) >= hora_actual
]

prob_lluvia = max(pp_filtrada) if pp_filtrada else 0


# Viento
if vty:
    # sacamos velocidad y dirección del último dict que contenga ambas claves
    viento = next((d for d in reversed(vty) if 'velocidad' in d and 'direccion' in d), {})
    vel_viento = viento.get('velocidad', [None])[0] if isinstance(viento.get('velocidad'), list) else viento.get('velocidad')
    dir_viento = viento.get('direccion', [None])[0] if isinstance(viento.get('direccion'), list) else viento.get('direccion')
else:
    vel_viento = dir_viento = None

# Fila día actual
fila_hoy = {
    "fecha": hoy['fecha'].split("T")[0],
    "tmax": tmax,
    "tmin": tmin,
    "estadoCielo": estado,
    "vel_viento": vel_viento,
    "dir_viento": dir_viento,
    "hummax": hummax,
    "hummin": hummin,
    "prob_lluvia": prob_lluvia  
}

# Insertar la fila en el dataframe semanal
clima_semana = clima_semana[clima_semana['fecha'] != fila_hoy['fecha']]
clima_semana = pd.concat([pd.DataFrame([fila_hoy]), clima_semana], ignore_index=True)

# Ordenar cronológicamente
clima_semana['fecha'] = pd.to_datetime(clima_semana['fecha'])
clima_semana = clima_semana.sort_values('fecha').reset_index(drop=True)

print(clima_semana)

       fecha  tmax  tmin         estadoCielo vel_viento dir_viento  hummax  \
0 2025-07-26    31    26           Despejado         15          S      46   
1 2025-07-27    37    19           Despejado         10         NO      65   
2 2025-07-28    38    20           Despejado          5          N      55   
3 2025-07-29    35    18         Poco nuboso         10          N      85   
4 2025-07-30    34    17         Poco nuboso         10         NE      90   
5 2025-07-31    35    17  Intervalos nubosos         15          N      65   
6 2025-08-01    36    18  Intervalos nubosos          0          C      60   

   hummin  prob_lluvia  
0      19            0  
1      15            0  
2      10            0  
3      20            5  
4      25            5  
5      15           20  
6      20           25  


### Visualización

In [19]:
def mostrar_prediccion_semanal(df: pd.DataFrame):
    dias = {'Mon':'lun','Tue':'mar','Wed':'mié','Thu':'jue',
            'Fri':'vie','Sat':'sáb','Sun':'dom'}

    cards = []
    for _, row in df.iterrows():
        ts = pd.to_datetime(row['fecha'])
        dia_abbr = dias.get(ts.strftime('%a'), ts.strftime('%a'))
        numero   = ts.day
        p_lluv   = int(row.get('prob_lluvia', 0))

        # Color de fondo según tipo de día
        if p_lluv >= 50:
            bg = "#e5f2ff"   # lluvioso
        elif p_lluv > 0:
            bg = "#f0f4f8"   # parcial
        else:
            bg = "#fffae5"   # soleado

        # Icono principal
        if p_lluv >= 50:
            main_icon = '🌧️'
        elif p_lluv > 0:
            main_icon = '⛅'
        else:
            main_icon = '☀️'

        card = f"""
        <div style="
            min-width: 120px;
            border-radius: .75rem;
            background-color: {bg};
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            text-align: center;
            padding: 1rem;
            font-family: sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: .5rem;
        ">
          <!-- Día -->
          <div style="font-weight:700; font-size:1.4rem; color:#333;">
            {dia_abbr} {numero}
          </div>

          <!-- Icono principal -->
          <div style="font-size:2rem; line-height:1;">
            {main_icon}
          </div>

          <!-- Temperaturas -->
          <div style="
                font-size:1.1rem;
                display:flex;
                align-items:center;
                gap:.25rem;
          ">            
            <span style="color:blue; font-weight:600; font-size:1.1;">{int(row['tmin'])}°</span>
            <span style="color:#666;">/</span>
            <span style="color:red; font-weight:600; font-size:1.1;">{int(row['tmax'])}°</span>
            
          </div>

          <!-- Fila de detalles: viento, humedad, lluvia -->
          <div style="
                display: flex;
                justify-content: space-around;
                width: 100%;
                font-size: .85rem;
                color: #333;
                margin-top: .5rem;
          ">
            <!-- Viento -->
            <div style="display:flex; flex-direction:column; align-items:center;">
              <div style="font-size:1.5rem;">💨</div>
              <div style="font-size:1.1rem;">{row['vel_viento']} m/s</div>
              <div style="color:#666; font-size:1rem;">{row['dir_viento']}</div>
            </div>
            <!-- Humedad -->
            <div style="display:flex; flex-direction:column; align-items:center;">
              <div style="font-size:1.5rem;">🌫️</div>
              <div style="font-size:1.1rem;">{int(row['hummin'])}–{int(row['hummax'])}%</div>
            </div>
            <!-- Prob. lluvia -->
            <div style="display:flex; flex-direction:column; align-items:center;">
              <div style="font-size:1.5rem;">☔</div>
              <div style="font-size:1.1rem;">{p_lluv}%</div>
            </div>
          </div>
        </div>
        """
        cards.append(card)

    html = f"""
    <div style="
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(120px, 1fr));
        gap: .75rem;
        overflow-x: auto;
        padding: .5rem;
    ">
      {''.join(cards)}
    </div>
    """
    display(HTML(html))

In [20]:
mostrar_prediccion_semanal(clima_semana)


## Predicción días próximos

### Obtención de datos

In [21]:
# Consulta predicción horaria
url_pred = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/horaria/{cod_muni}/"
headers = {"api_key": AEMET_API_KEY}

print(f"\nConsultando predicción horaria para municipio {cod_muni} ({municipio})")
r_pred = requests.get(url_pred, headers=headers)
r_pred.raise_for_status()
resp = r_pred.json()




Consultando predicción horaria para municipio 18027 (Benalua)


In [22]:
# Seguir enlace de datos si se puede
data_url = resp.get("datos")
if data_url:
    r_data = requests.get(data_url, headers=headers)
    r_data.raise_for_status()
    pred_data_horaria = r_data.json()
else:
    pred_data_horaria = resp

In [23]:
pred_data_horaria

[{'origen': {'productor': 'Agencia Estatal de Meteorología - AEMET. Gobierno de España',
   'web': 'https://www.aemet.es',
   'enlace': 'https://www.aemet.es/es/eltiempo/prediccion/municipios/horas/benalua-id18027',
   'language': 'es',
   'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
   'notaLegal': 'https://www.aemet.es/es/nota_legal'},
  'elaborado': '2025-07-26T14:28:09',
  'nombre': 'Benalúa',
  'provincia': 'Granada',
  'prediccion': {'dia': [{'estadoCielo': [{'value': '11',
       'periodo': '08',
       'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '09', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '10', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '11', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '12', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '13', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '14', '

### Formateo de datos

In [24]:
dias = pred_data_horaria[0]["prediccion"].get("dia", [])

dias

[{'estadoCielo': [{'value': '11', 'periodo': '08', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '09', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '10', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '11', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '12', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '13', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '14', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '15', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '16', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '17', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '18', 'descripcion': 'Despejado'},
   {'value': '12', 'periodo': '19', 'descripcion': 'Poco nuboso'},
   {'value': '11', 'periodo': '20', 'descripcion': 'Despejado'},
   {'value': '11', 'periodo': '21', 'descripcion': 'Despejado'},
   {'value': '11n', 'periodo': '22', 'descripcion': 'Despejado'},
   {'va

In [25]:
filas = []

print(f"\nPredicción 48 horas para municipio {cod_muni} ({municipio}):")
for dia in dias:
    # Dar formato a la fecha
    fecha_raw  = dia.get("fecha", "")
    fecha_base = fecha_raw.split("T")[0] if "T" in fecha_raw else fecha_raw

    # Un diccionario por hora

    # Precipitación acumulada (mm)
    precip_map = {
        d["periodo"]: float(d["value"])
        for d in dia.get("precipitacion", [])
    }

    # Humedad relativa (%)
    hum_map = {
        d["periodo"]: int(d["value"])
        for d in dia.get("humedadRelativa", [])
    }

    # Viento: velocidad y dirección
    wind_map = {}
    for v in dia.get("vientoAndRachaMax", []):
        if "direccion" in v and isinstance(v["direccion"], list):
            vel = int(v["velocidad"][0]) if isinstance(v["velocidad"], list) else int(v["velocidad"])
            direc = v["direccion"][0]
            wind_map[v["periodo"]] = {"vel": vel, "dir": direc}

    # Probabilidad de precipitación (%)
    prob_map = {}
    for p in dia.get("probPrecipitacion", []):
        per = p.get("periodo", "")
        val = int(p.get("value", 0))
        if len(per) == 4 and per.isdigit():
            h_start, h_end = int(per[:2]), int(per[2:])
            if h_end > h_start:
                hrs = range(h_start, h_end)
            else:
                # cruza medianoche
                hrs = list(range(h_start, 24)) + list(range(0, h_end))
            for h in hrs:
                prob_map[f"{h:02d}"] = val

    # Recorro cada registro horario de temperatura y creo la fila
    for temp_rec in dia.get("temperatura", []):
        hora = temp_rec.get("periodo", "").zfill(2)  # e.g. '09'
        fecha_hora = f"{fecha_base} {hora}:00"

        filas.append({
            "fecha"            : fecha_hora,
            "temperatura"      : int(temp_rec.get("value", 0)),
            "prob_lluvia"      : prob_map.get(hora, 0),
            "precipitacion"    : precip_map.get(hora, 0),
            "humedad"          : hum_map.get(hora, 0),
            "velocidad_viento" : wind_map.get(hora, {}).get("vel"),
            "direccion_viento" : wind_map.get(hora, {}).get("dir"),
        })

# Creamos el dataframe
clima_horaria = pd.DataFrame(filas)

if not clima_horaria.empty:
    print("\nClima horario:")
    print(clima_horaria)


Predicción 48 horas para municipio 18027 (Benalua):

Clima horario:
               fecha  temperatura  prob_lluvia  precipitacion  humedad  \
0   2025-07-26 09:00           21            0            0.0       74   
1   2025-07-26 10:00           23            0            0.0       63   
2   2025-07-26 11:00           26            0            0.0       54   
3   2025-07-26 12:00           28            0            0.0       46   
4   2025-07-26 13:00           30            0            0.0       36   
5   2025-07-26 14:00           32            0            0.0       26   
6   2025-07-26 15:00           32            0            0.0       24   
7   2025-07-26 16:00           33            0            0.0       21   
8   2025-07-26 17:00           34            0            0.0       21   
9   2025-07-26 18:00           33            0            0.0       21   
10  2025-07-26 19:00           32            0            0.0       20   
11  2025-07-26 20:00           31          

### Visualización

In [26]:
def mostrar_prediccion_horaria_prec_temp(df: pd.DataFrame):
    # Pasar 'fecha' a datetime
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d %H:%M")

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Precipitación
    fig.add_trace(
        go.Bar(
            x=df['fecha'],
            y=df['precipitacion'],
            name="Precipitación (mm)",
            marker=dict(color="rgba(30,144,255,0.6)"),
            hovertemplate="%{x|%d %b %H:%M}<br>Precip: %{y:.1f} mm<extra></extra>"
        ),
        secondary_y=False
    )

    # Temperatura
    fig.add_trace(
        go.Scatter(
            x=df['fecha'],
            y=df['temperatura'],
            name="Temperatura (°C)",
            mode="lines+markers",
            line=dict(color="crimson", width=4, shape="spline"),
            marker=dict(size=8),
            hovertemplate="%{x|%d %b %H:%M}<br>Temp: %{y:.1f}°C<extra></extra>"
        ),
        secondary_y=True
    )

    # Layout
    fig.update_layout(
        title=dict(text="<b>48 h: Precipitación · Temperatura</b>", 
                   x=0.02, xanchor="left"),
        legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
        template="plotly_white",
        margin=dict(l=40, r=40, t=60, b=40),
        hovermode="x unified"
    )

    # Ejes
    fig.update_xaxes(
        title_text="Fecha y hora",
        tickformat="%d %b<br>%H:%M",
        tickangle=-45,
        showgrid=True,
        gridcolor="rgba(200,200,200,0.2)",
        # una etiqueta cada 3 horas (3 * 3600000 ms)
        dtick= 3 * 3600 * 1000
    )
    fig.update_yaxes(
        title_text="Precipitación (mm)",
        secondary_y=False,
        showgrid=True,
        gridcolor="rgba(200,200,200,0.2)"
    )
    fig.update_yaxes(
        title_text="Temp (°C)",
        secondary_y=True,
        showgrid=False
    )

    return fig


In [27]:
fig = mostrar_prediccion_horaria_prec_temp(clima_horaria)
fig.show()
